In [22]:

import networkx
import pydot
import pandas as pd

graph = pydot.graph_from_dot_file('exp_short.gv')[0]

networkx_graph = networkx.drawing.nx_pydot.from_pydot(graph)

node_data = []

for node in networkx_graph.nodes:
    node_data.append({'node': node, **networkx_graph.nodes[node]})

node_data = pd.DataFrame(node_data)
node_data['is_leaf'] = node_data['label'].str.contains('Patient_3_Cell_')
node_data['label'] = node_data['label'].str.replace('"', '')
node_data['cell_id'] = None
node_data.loc[node_data['is_leaf'], 'cell_id'] = (
    node_data.loc[node_data['is_leaf'], 'label']
    .str.replace(r'.bam\\n', ''))

node_data.to_csv('exp_short_nodes.csv', index=False)


/Users/mcphera1/Projects/bicg_scwgs/SCIPhI/venv/lib/python3.7/site-packages/ipykernel_launcher.py:20: FutureWarning: The default value of regex will change from True to False in a future version.


In [80]:

# From: https://stackoverflow.com/questions/46444454/save-networkx-tree-in-newick-format

leaves = set()

def tree_to_newick(g, root=None):
    if root is None:
        roots = list(filter(lambda p: p[1] == 0, g.in_degree()))
        assert 1 == len(roots)
        root = roots[0][0]
    subgs = []
    for child in g[root]:
        if len(g[child]) > 0:
            subgs.append(tree_to_newick(g, root=child))
        else:
            leaves.add(child)
            subgs.append(child)
    return "(" + ','.join(subgs) + ")" + root

newick_str = tree_to_newick(networkx_graph) + ';'

with open('exp_short_tree.newick', 'w') as f:
    f.write(newick_str)
